## VGG
vggNet 是第一个真正意义上的深层网络结构，其是 ImageNet2014年的冠军，得益于 python 的函数和循环，我们能够非常方便地构建重复结构的深层网络。

vgg 的网络结构非常简单，就是不断地堆叠卷积层和池化层，下面是一个简单的图示

![](https://ws4.sinaimg.cn/large/006tNc79ly1fmpk5smtidj307n0dx3yv.jpg)

vgg 几乎全部使用 3 x 3 的卷积核以及 2 x 2 的池化层，使用小的卷积核进行多层的堆叠和一个大的卷积核的感受野是相同的，同时小的卷积核还能减少参数，同时可以有更深的结构。

vgg 的一个关键就是使用很多层 3 x 3 的卷积然后再使用一个最大池化层，这个模块被使用了很多次，下面我们照着这个结构来写一写

In [ ]:
import tensorflow as tf
from utils import cifar10_input

In [ ]:
# 我们定义一个批次有64个样本
batch_size = 64

In [ ]:
train_imgs, train_labels, val_imgs, val_labels = cifar10_input.load_data(data_dir='cifar10_data')

从`utils/`中可以使用我们之前定义过的各种接口

In [ ]:
from utils.layers import conv, max_pool, fc
from utils.learning import train

- `vgg`是一个不断堆叠的网络结构, 我们从它重复的最小单元写起

In [ ]:
def vgg_block(inputs, num_convs, out_depth, scope='vgg_block', reuse=None):
    """构建vgg_block.
    
    一个 vgg_block 由`num_convs`个卷积层和一个最大值池化层构成.
    
    Args:
        inputs: 输入
        num_convs: 这一个block里卷积层的个数
        out_depth: 每一个卷积层的卷积核个数
        scope: 变量域名
        reuse: 是否复用
    """
    in_depth = inputs.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = inputs
        # 循环定义`num_convs`个卷积层
        for i in range(num_convs):
            net = conv(net, ksize=[3, 3], out_depth=out_depth, strides=[1, 1], padding='SAME', scope='conv%d' % i, reuse=reuse)
        net = max_pool(net, [2, 2], [2, 2], name='pool')
        
        return net

- 然后我们把很多个不同的`vgg_block`堆叠在一起

In [ ]:
def vgg_stack(inputs, num_convs, out_depths, scope='vgg_stack', reuse=None):
    """构建vgg_stack.
    
    一个 vgg_stack 将若干个不同的`vgg_block`进行`stack`(堆叠)
    
    Args:
        inputs: 输入
        num_convs: 每一个block里卷积层的个数, 列表. 如`[1, 2, 3]`
        out_depths: 每一个block的卷积核个数, 列表, 如`[64, 128, 256]`
        scope: 变量域名
        reuse: 是否复用
    """
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = inputs
        for i, (n, d) in enumerate(zip(num_convs, out_depths)):
            net = vgg_block(net, n, d, scope='block%d' % i)
        return net

- 最后我们在通过几个全连接层将`vgg`搭建完成

In [ ]:
def vgg(inputs, num_convs, out_depths, num_outputs, scope='vgg', reuse=None):
    """构建vgg.
    
    一个 vgg 先经过`vgg_stack`后再连接两个全连接层.
    
    Args:
        inputs: 输入
        num_convs: 每一个 vgg_block 的卷积层的个数
        out_depths: 每一个 vgg_block 卷积核个数
        num_outputs: 最后输出向量的维数
        scope: 变量域名
        reuse: 是否复用
    """
    with tf.variable_scope(scope, reuse=reuse) as sc:
        net = vgg_stack(inputs, num_convs, out_depths)
        with tf.variable_scope('classification'):
            net = tf.reshape(net, (batch_size, -1))
            net = fc(net, 100, scope='fc1')
            net = fc(net, num_outputs, act=tf.identity, scope='classification')
        
        return net

In [ ]:
train_out = vgg(train_imgs, (1, 1, 2, 2, 2), (64, 128, 256, 512, 512), 10)
# 复用上面的参数
val_out = vgg(val_imgs, (1, 1, 2, 2, 2), (64, 128, 256, 512, 512), 10, reuse=True)

#### 定义损失计算`op`

In [ ]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

#### 定义正确率计算`op`

In [ ]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

#### 构建训练`op`

In [ ]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)
train_op = opt.minimize(train_loss)

#### 开始训练

In [ ]:
train(train_op, train_loss, train_acc, val_loss, val_acc, 20000, batch_size)

可以看到, 20000步的训练后, `VGG`在训练集和测试集分别达到了`0.97`和`0.75`的准确率

### 小练习
尝试用`Keras`或者`tf-slim`来实现`vgg`网络并训练